<a href="https://colab.research.google.com/github/Nnamaka/ML_specializations/blob/main/Computer_vision/faceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib
!pip install wget

##Import Dependencies

In [2]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

##Create Folder Structures


In [4]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
# make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

#Collect Positive and Anchors.   
And preprocess them

  
My positive and anchor images were stored in my Google Drive.

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [7]:
!cp "/content/drive/MyDrive/TFOD images/face.tar.gz" /content && cp "/content/drive/MyDrive/TFOD images/anchor.tar.gz" /content

uncompressed data file of positive image

In [8]:
!tar -xzf face.tar.gz && tar -xzf anchor.tar.gz

resize and store positive and anchor images in data folder, in their appropriate folders.

In [9]:
width = 250
height = 250
dim = (width, height)


In [10]:
anch_pos = ["/content/Anchor/","/content/Myimages"]

for path in anch_pos:
  for filename in os.listdir(path):
    if filename.endswith('.jpg'):
      try:
        if 'Anchor' in path:
          img = cv2.imread("/content/Anchor/" + filename, cv2.IMREAD_UNCHANGED)
          new_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
          cv2.imwrite(ANC_PATH + '/' + filename, new_img)

        if 'Myimages' in path:
          img = cv2.imread("/content/Myimages/" + filename, cv2.IMREAD_UNCHANGED)
          new_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
          cv2.imwrite(POS_PATH + '/' + filename, new_img)
      except:
        pass
      

Augument all positive and anchor images to increase data size and possible expose the model to newer samples

In [11]:
def augment(img):
  data = []
  for i in range(15):
    img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
    img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
    img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100), np.random.randint(100)))
    img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100),np.random.randint(100)))
    img = tf.image.stateless_random_saturation(img, lower=0.9,upper=1, seed=(np.random.randint(100),np.random.randint(100)))

    data.append(img)

  return data

In [12]:
import os
import uuid

In [13]:
paths = [POS_PATH, ANC_PATH]

for path in paths:
  print("inside " + path)
  for file_name in os.listdir(os.path.join(path)):
    img_path = os.path.join(path, file_name)
    img = cv2.imread(img_path)
    augmented_images = augment(img) 
    
    for image in augmented_images:
        cv2.imwrite(os.path.join(path, '{}.jpg'.format(uuid.uuid1())), image.numpy())

inside data/positive
inside data/anchor


check the no of positive and anchor images

In [14]:
for path in paths:
  print("no of images in " + path + " : ", end="")
  a = 0
  for image in os.listdir(os.path.join(path)):
    if image.endswith('.jpg'):
      a = a + 1
  print(a)

no of images in data/positive : 480
no of images in data/anchor : 480


#Collect Negative Images

In [15]:
!wget -O wildImages.tar http://vis-www.cs.umass.edu/lfw/lfw.tgz

--2022-09-23 21:40:04--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘wildImages.tar’

wildImages.tar      100%[===================>] 172.20M  31.2MB/s    in 6.2s    

2022-09-23 21:40:10 (28.0 MB/s) - ‘wildImages.tar’ saved [180566744/180566744]



###Untar Labelled Faces in the wild Dataset



In [16]:
!tar -xzf wildImages.tar

In [17]:
# move ifw images to the negative folder image directory
# a = 0
# no_of_neg_img = 450
for directory in os.listdir('lfw'):
  for file in os.listdir(os.path.join('lfw', directory)):
    # a = a + 1;

    # if ( a == no_of_neg_img):
    #   break

    old_path = os.path.join('lfw', directory, file)
    new_path = os.path.join(NEG_PATH, file)
    os.replace(old_path, new_path)

#Load and Preprocess Images

##Get image directories

In [20]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(450)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(450)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(450)


In [21]:
dir_test = anchor.as_numpy_iterator()

In [24]:
print(dir_test.next())

b'data/anchor/IMG_20220920_152033_954.jpg'


#Preprocessing - Scale and Resize

In [27]:
def preprocess(file_Path):

  byte_img = tf.io.read_file(file_Path)

  img = tf.io.decode_jpeg(byte_img)

  img = tf.image.resize(img, (100, 100))
  img = img / 255.0

  return img

In [29]:
img = preprocess('data/anchor/IMG_20220920_152033_954.jpg')


In [30]:
img.numpy().max() 


1.0